# Versión tres del modelo (Regresión logistica)
Voy a reoordenar el dataframe del train a  a ver si ni me peta,

In [1]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression # el modelo
from sklearn.metrics import accuracy_score, classification_report # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int



In [2]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [3]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [4]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [5]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = 'home'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'tie'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[0]), "result"] = 'visitor'

In [6]:
df_games.drop(["time", "date", "score"], axis=1, inplace=True)  

In [13]:
df_home = df_games[["season", "division", "matchday", "home_team", "result"]].rename(columns={"home_team":"team"})
df_visitor=df_games[["season", "division", "matchday", "away_team", "result"]].rename(columns={"away_team":"team"})
df2 = pd.concat([df_home, df_visitor])
df2 = df2.drop_duplicates()

,season,division,matchday,team,result
0,1928-1929,1,1,Arenas Club,None
1,1928-1929,1,1,Espanyol,home
2,1928-1929,1,1,Real Madrid,home
3,1928-1929,1,1,Donostia,tie
4,1928-1929,1,1,Racing,None
...,...,...,...,...,...
48775,2021-2022,2,42,UD Ibiza,None
48776,2021-2022,2,42,Real Zaragoza,None
48777,2021-2022,2,42,UD Las Palmas,None
48778,2021-2022,2,42,FC Cartagena,None


: 

# Empieza el modelo (Logistic regression)
Nuestro modelo/problema es de tipo clasificación. La predicción nos tiene que dar uno de los tres valores, NO es una variable continua.

Ya tenemos los datos. Ahora hay que definir nuestras variables. X son los datos que vamos a tener en cuenta, e Y es la "variable etiqueta", es decir, la que tenemos que predecir. La etiqueta tiene que ser de tipo int, empezamos con eso.

In [8]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "2000:2001"


In [9]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0])]


In [10]:
df_games_train = df2
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_train["season"] = df_games_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_train = df_games_train[(df_games_train["season"].str.split("-").str[0] >= season_train.split(":")[0])]


In [11]:
# Hacemos el merge del dataframe para train
df_train = pd.merge(df_games_train, df_teams_train, left_on='team', right_on='team', how='left', suffixes=("_home", ""))
df_train = pd.merge(df_train, df_teams_train, left_on='team', right_on='team', how='left', suffixes=("_away",""))

MemoryError: Unable to allocate 141. GiB for an array with shape (18936148384,) and data type int64

In [ ]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])
df_train["home"] = le.fit_transform(df_train["team"])

: 

In [ ]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

: 

In [ ]:
df_teams_test = df_teams[(df_teams["season"] == season) & (df_teams["division"] == division) & (df_teams["matchday"] == matchday)]
df_games_test = df2[(df2["season"] == season) & (df2["division"] == division) & (df2["matchday"] == matchday)]

: 

In [ ]:
# Hacemos el merge del dataframe para hacer el test
df_test = pd.merge(df_games_test, df_teams_test, left_on='team', right_on='team', how='left', suffixes=("_home", ""))
df_test = pd.merge(df_test, df_teams_test, left_on='team', right_on='team', how='left', suffixes=("_away",""))

: 

In [ ]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])
df_test["home"] = le.fit_transform(df_test["team"])

: 

In [ ]:
X_test = df_test[['rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

: 

In [ ]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = LogisticRegression()
model.fit(X_train, y_train)

: 

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

: 

In [ ]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

: 

In [ ]:
# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)']))

: 